## Important

in the nav2_params.yaml files:  
  
for the global_costmap, the global_frame has to be map!!!  
  
<pre> global_costmap: 
    global_costmap:
        ros__parameters:
            update_frequency: 1.0
            publish_frequency: 1.0 
            global_frame: map           <---
            
</pre>

---

### These are the things tested to get global and local costmaps to appear in rqt_graph from map

| test                                                      | additional tests                                      | costmaps  in rqt_graph          |results|
|----------                                                 |----------                                             |----------         |----------|
|regular withouth changes||no|robot move with send_goals|
| set all occurances of map to /map in .yaml file       |  |no|[bt_navigator-4] [ERROR] [1758198093.200106623] [transformPoseInTargetFrame]: Failed to transform from robot_agent_1_base_link to /map [bt_navigator-4] [ERROR] [1758198093.200198721] [robot_agent_1.bt_navigator]: Initial robot pose is not available. [bt_navigator-4] [WARN] [1758198093.200269129] [robot_agent_1.bt_navigator]: [navigate_to_pose] [ActionServer] Aborting handle.         |
|                                                           | removing all remappings of /map in nav2.launch.py          |no| [planner_server-2] [WARN] [1758197545.824340385] [robot_agent_1.global_costmap.global_costmap]: Can't update static costmap layer, no map received                  |
||removed all remappings of tf and tf_static|no|[WARN] [1758197752.999925492] [robot_agent_1.bt_navigator.rclcpp_action]: Failed to send goal response 044305a1-2f77-487d-935e-475da68e4bd8 (timeout): client will not receive response, at ./src/rmw_response.cpp:153, at ./src/rcl/service.c:400|
|changed plugins: ["voxel_layer", "inflation_layer"] to plugins: ["static_layer", "voxel_layer", "inflation_layer"] in local_costmap: yaml||no but subscribed to in ros2 topic info /map|robot move with send_goals|
|old .yaml file from main|||[planner_server-2] [FATAL] [1758199835.226323747] [robot_agent_1.planner_server]: Failed to create global planner. Exception: According to the loaded plugin descriptions the class nav2_navfn_planner/NavfnPlanner with base class type nav2_core::GlobalPlanner does not exist. Declared types are  nav2_navfn_planner::NavfnPlanner nav2_smac_planner::SmacPlanner2D nav2_smac_planner::SmacPlannerHybrid nav2_smac_planner::SmacPlannerLattice nav2_theta_star_planner::ThetaStarPlanner|
||old .yaml file with new planner_server||[bt_navigator-4] [FATAL] [1758199961.035270325] [robot_agent_1.bt_navigator]: Failed to create navigator id navigate_to_pose. Exception: ID [ComputePathToPose] already registered [bt_navigator-4] [INFO] [1758199961.035378473] [robot_agent_1.bt_navigator]: Cleaning up [bt_navigator-4] [INFO] [1758199961.060250559] [robot_agent_1.bt_navigator]: Completed Cleaning up [lifecycle_manager-5] [ERROR] [1758199961.060761666] [robot_agent_1.lifecycle_manager_navigation]: Failed to change state for node: bt_navigator [lifecycle_manager-5] [ERROR] [1758199961.060846783] [robot_agent_1.lifecycle_manager_navigation]: Failed to bring up all requested nodes. Aborting bringup.|
  
  
  
---

## repeatability of nav2 pipeline

here is a check of the repeatability of the nav2 pipeline to se what costmaps are launching.  
start 4 new terminals each try
***
#### terminal 1:
./control.sh kill  
./control.sh build  
./control.sh gpss  
#### terminal 2:
./control.sh map
#### terminal 3:
./control.sh nav
#### terminal 4:
rqt_graph
    
#### the results are the following:  

|try|costmaps|
|---|---|
|1|local_costmap|
|2| no costmap|
|3|local_costmap|
|4|map_server didnt launch properly, still global_costmap subscribed to /map|
|5|[bt_navigator-4] [ERROR] [1758202422.295292030] [robot_agent_1.bt_navigator_navigate_to_pose_rclcpp_node]: "follow_path" action server not available after waiting for 1.00s [bt_navigator-4] [ERROR] [1758202422.324267825] [robot_agent_1.bt_navigator]: Exception when loading BT: Action server follow_path not available [bt_navigator-4] [ERROR] [1758202422.324424311] [robot_agent_1.bt_navigator]: Error loading XML file: /opt/ros/jazzy/share/nav2_bt_navigator/behavior_trees/navigate_to_pose_w_replanning_and_recovery.xml [lifecycle_manager-5] [ERROR] [1758202422.325205141] [robot_agent_1.lifecycle_manager_navigation]: Failed to change state for node: bt_navigator [lifecycle_manager-5] [ERROR] [1758202422.325309182] [robot_agent_1.lifecycle_manager_navigation]: Failed to bring up all requested nodes. Aborting bringup. [controller_server-1] [WARN] [1758202422.901179045] [robot_agent_1.local_costmap.local_costmap]: Can't update static costmap layer, no map received still global_costmap subscribed to /map|
|6|no costmap|
|7|[planner_server-2] [WARN] [1758203073.181270363] [robot_agent_1.planner_server.rclcpp]: failed to send response to /robot_agent_1/planner_server/change_state (timeout): client will not receive response, at ./src/rmw_response.cpp:153, at ./src/rcl/service.c:400 still global_costmap|
|8|local_costmap|